**Install Spark**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.mirrors.pair.com/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

!tar -xvf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

**Task 1 - Step 1**

In [ ]:
import pandas as pd
from tabulate import tabulate
from pyspark import SparkContext
import datetime
import pyspark as sp


sc = SparkContext


nSlices = 10
sc = sp.SparkContext
rdd1 = sc.parallelize([1],nSlices)

# [...]
sc.close()

ModuleNotFoundError: ignored

In [ ]:
#read CSV into a DF
df = spark.read.format('csv').options(header='true', multiline='true').load('Amazon_Responded_Oct05.csv')

#df.show()
df.count()


397583

In [ ]:
#include only the six columns needed 
df = df[['id_str','tweet_created_at','user_verified','favorite_count','retweet_count','text_']]

#Drop na values
df = df.dropna()

df.count()
#df.show()

348193

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, functions as F

In [ ]:
#Create a new column which contains month and date 
df = df.withColumn("date", F.regexp_extract(df.tweet_created_at, r'(\b\w{3} \d{2})',0))
df.show()

In [ ]:
#Step 1 - Remove the records where “user_verified” is “FALSE” i.e include records which is equal to True
df = df[df.user_verified == "True"]

df.show()
df.count()

**Task 1 - Step 2**

In [ ]:
#convert the df to RDD
rdd = df.rdd
rdd.collect()


In [ ]:
step2 = rdd.map(lambda x: (x[6], x[4])).groupByKey().mapValues(lambda vals: len(vals)).collect()

number_of_tweets_per_day = spark.createDataFrame(step2)

number_of_tweets_per_day.orderBy('_2', ascending=False).show(10)

+------+----+
|    _1|  _2|
+------+----+
|Jan 03|1536|
|Jan 10|1508|
|Jan 11|1496|
|Jan 12|1409|
|Jan 06|1363|
|Jan 07|1360|
|Jan 20|1342|
|Mar 02|1298|
|Jan 13|1295|
|Jan 21|1292|
+------+----+
only showing top 10 rows



Task 2

Amazon Data

In [ ]:
#include only the two columns needed 
df2 = df[['id_str','text_']]
df2.show()
df2.count()



Find Text Data

In [ ]:
#read CSV into a DF
find_text=spark.read.format('csv').options(header='true', multiline='true').load('find_text.csv')
find_text.show()
find_text.count()


 Convert to RDD

In [ ]:
Amazon=df2.rdd
find_text1=find_text.rdd
find_text1.collect()

Join 

In [ ]:
op=Amazon.join(find_text)
#x=Amazon
#y=find_text1
#a=sorted(x.leftOuterJoin(y).collect())
#stopwords = ['None']
#rdd3 = a.filter(lambda x: x not in stopwords)
#a=(list(filter(lambda x: x[0]>0,x[2] > 0)))
#a=(list(filter(None)))


#op.collect()
#op=Amazon.map((lambda x:(x[0]),(x[1])).leftjoin(find_text(lambda x:(x[0]),x[1]))).collect()



Convert to Data Frame

In [ ]:
from pyspark.sql.types import*
schema = StructType([StructField("id_str", StringType(), True), StructField("text", StringType(), True)])
DF1=spark.createDataFrame(op,schema=schema)
DF1.show()


#df = spark.createDataFrame(a,['idx', 'tuple'],schema=schema)
#df.show()

Convert to CSV

In [ ]:
DF1.toPandas().to_csv('FIND_TEXT3.csv') #refresh and save the text file


In [ ]:
DF1.toPandas().to_csv('FIND_TEXT1.csv')
#DF1.write.format('com.databricks.spark.csv').save('FIND_TEXT.csv')

In [ ]:
dfjoin=df2.join(find_text, df2.id_str==find_text.id_str, how='left')
dfjoin.show()